### Importing the Libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
tf.__version__ 

'2.19.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [7]:
training_set = train_datagen.flow_from_directory(r'C:\Users\Shubhrojit Panda\Desktop\Sign-Language-To-Text-Conversion\dataSet\trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [9]:
test_set = test_datagen.flow_from_directory(r'C:\Users\Shubhrojit Panda\Desktop\Sign-Language-To-Text-Conversion\dataSet\testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [10]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [11]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

c:\Users\Shubhrojit Panda\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Step 2 - Pooling

In [12]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [13]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [14]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [15]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [16]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 27)             │         1,755 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,224,347 (16.11 MB)

 Trainable params: 4,224,347 (16.11 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5


c:\Users\Shubhrojit Panda\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1285/1285 ━━━━━━━━━━━━━━━━━━━━ 317s 247ms/step - accuracy: 0.1834 - loss: 2.7564 - val_accuracy: 0.8104 - val_loss: 0.5287
Epoch 2/5
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 31s 24ms/step - accuracy: 0.6896 - loss: 0.9355 - val_accuracy: 0.9482 - val_loss: 0.1548
Epoch 3/5
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - accuracy: 0.8126 - loss: 0.5569 - val_accuracy: 0.9822 - val_loss: 0.0564
Epoch 4/5
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - accuracy: 0.8653 - loss: 0.3987 - val_accuracy: 0.9850 - val_loss: 0.0450
Epoch 5/5
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 30s 23ms/step - accuracy: 0.8930 - loss: 0.3295 - val_accuracy: 0.9923 - val_loss: 0.0265


#### Saving the Model

In [20]:
model_json = classifier.to_json()
with open("model_last.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_last.weights.h5')
print('Weights saved')

Model Saved
Weights saved
